In [33]:
# import required libraries
from azure.identity import DefaultAzureCredential
from azure.ai.ml import UserIdentityConfiguration, MLClient, Input, Output, load_component
from azure.ai.ml.dsl import pipeline
from azure.ai.ml.entities import Environment
from azure.ai.ml.constants import AssetTypes, InputOutputModes
from azure.ai.ml.parallel import parallel_run_function, RunFunction
from dotenv import load_dotenv, find_dotenv
from datetime import datetime
import os
import mlflow

In [3]:
# Initialize ML Client
load_dotenv(find_dotenv(), override=True)

# authenticate
credential = DefaultAzureCredential(tenantid=os.environ.get('TENANT_ID'))

# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id = os.environ.get('SUBSCRIPTION_ID'),
    resource_group_name = os.environ.get('RESOURCE_GROUP_NAME'),
    workspace_name = os.environ.get('WORKSPACE_NAME'),
)

print(dict(os.environ))

{'USER': 'zacksoenen', 'VSCODE_WSL_EXT_LOCATION': '/mnt/c/Users/zacksoenen/.vscode/extensions/ms-vscode-remote.remote-wsl-0.81.8', 'WT_PROFILE_ID': '{51855cb2-8cce-5362-8f54-464b92b32386}', 'SHLVL': '2', 'HOME': '/home/zacksoenen', 'OLDPWD': '/mnt/c/Users/zacksoenen/AppData/Local/Programs/Microsoft VS Code', 'DBUS_SESSION_BUS_ADDRESS': 'unix:path=/run/user/1000/bus', 'WSL_DISTRO_NAME': 'Ubuntu', 'WAYLAND_DISPLAY': 'wayland-0', 'LOGNAME': 'zacksoenen', 'PULSE_SERVER': 'unix:/mnt/wslg/PulseServer', 'WSL_INTEROP': '/run/WSL/97485_interop', 'NAME': 'Code', '_': '/home/zacksoenen/miniconda3/envs/mm_remote/bin/python', 'TERM': 'xterm-color', 'PATH': '/home/zacksoenen/miniconda3/envs/mm_remote/bin:/home/zacksoenen/.vscode-server/bin/0ee08df0cf4527e40edc9aa28f4b5bd38bbff2b2/bin/remote-cli:/home/zacksoenen/miniconda3/envs/mm_remote/bin:/home/zacksoenen/miniconda3/condabin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/usr/lib/wsl/lib:/mnt/c/Program Fil

In [4]:
# Use remote environment created in step 1
custom_env_name = "mm-remote-env-py37"

In [5]:
# Acess data asset
data_name = "oj-sim-sales-test"
test_data_asset = ml_client.data.get(data_name, label='latest')
print(test_data_asset.path)

azureml://subscriptions/9a729243-1221-42c5-824c-9e44cb2da98d/resourcegroups/many-models-rg/workspaces/mm-aml-wksp/datastores/workspaceblobstore/paths/LocalUpload/724411e17666e66d19b8612b775b7707/test_subset.csv


In [6]:
# Load data preparation component
partition_data = load_component(source="../src/components/partition_data/partition_data.yaml")

In [7]:
cpu_compute_target = "mm-cpu-cluster"

In [8]:
azureml_tracking_uri = ml_client.workspaces.get(ml_client.workspace_name).mlflow_tracking_uri

In [35]:
run_time = datetime.today().strftime('%Y%m%d%H%M%S')

output_dir = f"many_model_outputs/{run_time}/"
print(output_dir)

many_model_outputs/20240108144658/


In [50]:
# Create parallel inference step
# Declare parallel job, with a run_function task
many_model_inference_with_partition_keys = parallel_run_function(
    name="distributed_inference",
    display_name="Many Model Predictions",
    description="parallel job to batch predict with many models",
    inputs=dict(
        data_source=Input(
            type=AssetTypes.MLTABLE,
            description="Input mltable with predefined partition format.",
            mode=InputOutputModes.DIRECT,  # [Important] To use 'partition_keys', input MLTable is required to use 'direct' mode.
        ),
        drop_cols=Input(
            type="string",
            description="Columns need to be dropped before training. Split by comma.",
        ),
        experiment_names=Input(
            type="string",
            description="Name of training experiement(s) to be used to select best model. Split by comma.",
        ),
        metric_name=Input(
            type="string",
            description="Name of metric to be used to select best model, to be used in mlflow query. i.e. test_remse",
        ),
        date_col=Input(
            type="string",
            description="Name of date column in data",
        ),
        tracking_uri=Input(
            type="string",
            description="tracking uri of mlflow server (aml workspace)",
        ),
    ),
    outputs=dict(
        output_dir=Output(
            type=AssetTypes.URI_FOLDER,
            mode=InputOutputModes.RW_MOUNT,
        )
    ),
    input_data="${{inputs.data_source}}",  # Define which input data will be splitted into mini-batches
    partition_keys=["Store","Brand"],  # Use 'partition_keys' as the data division method. This method requires MLTable input with partition setting pre-defined in MLTable artifact.
    instance_count=6,  # Use X nodes from compute cluster to run this parallel job.
    max_concurrency_per_instance=2,  # Create 2 worker processors in each compute node to execute mini-batches.
    error_threshold=-1,  # Monitor the failures of item processed by the gap between mini-batch input count and returns. 'Many model training' scenario doesn't fit this setting and '-1' means ignore counting failure items by mini-batch returns.
    mini_batch_error_threshold=5,  # Monitor the failed mini-batch by exception, time out, or null return. When failed mini-batch count is higher than this setting, the parallel job will be marked as 'failed'.
    retry_settings=dict(
        max_retries=1,  # Define how many retries when mini-batch execution is failed by exception, time out, or null return.
        timeout=60,  # Define the timeout in second for each mini-batch execution.
    ),
    logging_level="DEBUG", # DEBUG, INFO, WARNING, ERROR, ETC
    task=RunFunction(
        code="../src/",
        entry_script="parallel_inference.py",
        environment=ml_client.environments.get(custom_env_name, label="latest"),
        program_arguments="--drop_cols ${{inputs.drop_cols}} "  # Passthrough input parameters into parallel_train script.
        "--experiment_names ${{inputs.experiment_names}} "
        "--metric_name ${{inputs.metric_name}} "
        "--tracking_uri ${{inputs.tracking_uri}} "
        "--date_col ${{inputs.date_col}} "
        "--output_dir ${{outputs.output_dir}} ",
    ),
)

In [51]:
# Build Pipeline
# Declare the overall input of the job.
test_oj_data = Input(
    path=test_data_asset.path,
    type=AssetTypes.URI_FILE,
    mode=InputOutputModes.RO_MOUNT,
)

# Declare pipeline structure.
@pipeline(display_name="parallel inference - outpath")
def parallel_inference_pipeline(pipeline_input_data):
    
    # Declare 1st data partition command job.
    partition_job = partition_data(
        data_source=pipeline_input_data,
        partition_keys="Store,Brand",
    )

    # Declare 2nd parallel model training job.
    parallel_inference = many_model_inference_with_partition_keys(
        data_source=partition_job.outputs.tabular_output_data,
        drop_cols="Advert,Store,Brand",
        experiment_names="many-models-parallel-training-job-mlflow-full",
        metric_name="test_rmse",
        date_col="WeekStarting",
        tracking_uri=azureml_tracking_uri
    )
    
    # User could override parallel job run-level property when invoke that parallel job/component in pipeline.
    # parallel_train.resources.instance_count = 5
    # parallel_train.max_concurrency_per_instance = 2
    # parallel_train.mini_batch_error_threshold = 10

# Create pipeline instance
inference_pipeline = parallel_inference_pipeline(pipeline_input_data=test_oj_data,)

# Set pipeline level compute
inference_pipeline.settings.default_compute = cpu_compute_target
print(inference_pipeline)

display_name: parallel inference - outpath
type: pipeline
inputs:
  pipeline_input_data:
    mode: ro_mount
    type: uri_file
    path: azureml://subscriptions/9a729243-1221-42c5-824c-9e44cb2da98d/resourcegroups/many-models-rg/workspaces/mm-aml-wksp/datastores/workspaceblobstore/paths/LocalUpload/724411e17666e66d19b8612b775b7707/test_subset.csv
jobs:
  partition_job:
    type: command
    inputs:
      data_source:
        path: ${{parent.inputs.pipeline_input_data}}
      partition_keys: Store,Brand
    component:
      $schema: https://azuremlschemas.azureedge.net/latest/commandComponent.schema.json
      name: partition_data
      version: '1'
      display_name: Partition data by keys
      type: command
      inputs:
        data_source:
          type: uri_file
        partition_keys:
          type: string
      outputs:
        tabular_output_data:
          type: mltable
      command: python partition_data.py --data_source ${{inputs.data_source}} --partition_keys
        ${{

In [52]:
# Run Inference Pipeline
pipeline_job = ml_client.jobs.create_or_update(
    inference_pipeline,
    experiment_name="many-models-parallel-inference-job",
)

Uploading src (0.01 MBs): 100%|██████████| 12062/12062 [00:00<00:00, 56374.10it/s]




In [21]:
# Coalesce outputs and save as ML data asset # Maybe not needed?

In [12]:
# Create Batch Endpoint 
# https://github.com/Azure/azureml-examples/blob/main/sdk/python/jobs/pipelines/1k_demand_forecasting_with_pipeline_components/automl-forecasting-demand-many-models-in-pipeline/automl-forecasting-demand-many-models-in-pipeline.ipynb

In [ ]:
# Create Deployment

In [ ]:
# Invoke Batch endpoint

In [19]:
mlflow_client = mlflow.tracking.MlflowClient()
azureml_tracking_uri = ml_client.workspaces.get(ml_client.workspace_name).mlflow_tracking_uri
mlflow.set_tracking_uri(azureml_tracking_uri)

reg = mlflow.sklearn.load_model(model_uri=f"models:/1000_dominicks/latest")

In [27]:
import pandas as pd
date_col = "WeekStarting"
input_data = pd.read_csv("../data/oj_sim_sales/test_subset.csv")
drop_cols = ["Advert", "Store", "Brand"]

# Prep Data
input_data[date_col] = pd.to_datetime(input_data[date_col])
input_data = input_data.set_index(date_col).sort_index(ascending=True)
input_data = input_data.assign(Week_Year=input_data.index.isocalendar().week.values)

X_test = input_data.drop(columns=drop_cols, errors="ignore")

# Make prediction
predictions = reg.predict(X_test)

# Combine prediction with input_data
output_data = input_data.copy()
output_data['predictions'] = predictions

display(input_data.head(10))

<class 'numpy.ndarray'>


,Store,Brand,Advert,Price,Week_Year,predictions
WeekStarting,,,,,,
1992-07-02,1027,dominicks,1,1.94,27,32732.850718
1992-07-02,1017,dominicks,1,2.57,27,34955.292829
1992-07-02,1025,tropicana,1,2.18,27,33124.575813
1992-07-02,1007,dominicks,1,2.00,27,23280.121954
1992-07-02,1012,tropicana,1,2.30,27,31458.285026
1992-07-02,1023,tropicana,1,2.24,27,34618.230204
1992-07-02,1025,minute_maid,1,2.02,27,30233.905537
1992-07-02,1024,tropicana,1,2.19,27,30871.927871
1992-07-02,1005,minute_maid,1,2.24,27,34618.230204


In [49]:
"many-models-parallel-training-job-mlflow-full".split(',')

['many-models-parallel-training-job-mlflow-full']